In [2]:
import numpy as np
import pandas as pd
import glob
from astropy.table import Table
import matplotlib.pyplot as plt
import json 
import collections
import astropy

In [11]:
all_SEDM = Table.read("D:/AllSNe/sample/SEDM_ML_sample.ascii", format = "ascii")

In [13]:
np.unique(all_SEDM["col1"])

AT2018hrg
SN2017iuk
ZTF17aaajnki
ZTF17aaaobyl
ZTF17aaapufz
ZTF17aaaukqn
ZTF17aaawgkc
ZTF17aaayjzy
ZTF17aaazdba
ZTF17aabcpcp
ZTF17aaburzk


In [3]:
zooniverse_results = Table.from_pandas(pd.read_csv("zooniverse_results_ZTFI.csv"))
superfit_results = Table.from_pandas(pd.read_csv(r"C:\Users\20xha\Documents\Caltech\Research\superfit\superfit\superfit_classes_ZTFI.csv"))

In [10]:
combined = astropy.table.join(superfit_results, zooniverse_results, keys = ["Version"])

In [11]:
TypeII_Final_Results = Table.from_pandas(pd.read_csv("TypeII_Final.csv"))

In [13]:
final_rcf_table = Table.from_pandas(pd.read_hdf(r"C:\Users\20xha\Documents\Caltech\Research/final_rcf_table.h5"))

In [15]:
TypeII_Final_Results

ZTF_Name,spectra_file,sn_type,superfit_sn_type,zooniverse_sn_type,z_sn,in_rcfI,sn_type_rcfI,in_marshall,sn_type_marshall
str12,str39,str2,str8,str7,float64,bool,str6,bool,str8
ZTF18aabssth,ZTF18aabssth_20180309_P60_v1.ascii,II,II,IIb,0.0374,True,II,True,II
ZTF18aabxlsv,ZTF18aabxlsv_20181110_Keck1_v1.ascii,II,IIb,IIn,0.0497,True,Ib,True,Ib
ZTF18aabybkt,ZTF18aabybkt_20180321_APO_v1.ascii,II,II,II-norm,0.0074,False,--,True,II
ZTF18aaccmnh,ZTF18aaccmnh_20180517_P200_v1.ascii,II,II,II-norm,0.0316,False,--,True,II
ZTF18aaeoiwu,ZTF18aaeoiwu_20191019_P60_v1.ascii,II,IIn,II-norm,0.6259,False,--,True,star
ZTF18aagpzjk,ZTF18aagpzjk_20180410_P200_v1.ascii,II,II,II-norm,0.0114,True,II,True,II
ZTF18aahfqbc,ZTF18aahfqbc_20180411_APO_v1.ascii,II,II,II-norm,0.0011,True,II,True,II
ZTF18aahnnly,ZTF18aahnnly_20180508_P200_v1.ascii,II,IIn,II-norm,0.0291,False,--,True,II
ZTF18aahqfee,ZTF18aahqfee_20180513_DCT_v1.ascii,II,II,II-norm,0.039,False,--,True,IIP


In [19]:
combined["ZTF_Name"]

AT2018hrg
SN2017iuk
ZTF17aaajnki
ZTF17aaajnki
ZTF17aaapufz
ZTF17aaaukqn
ZTF17aaawgkc
ZTF17aaayjzy
ZTF17aaazdba
ZTF17aaazdba
ZTF17aaazdba
